In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

from keras.optimizers import Adam



def build_discriminator():
    model = Sequential()
    model.add(Dense(41, input_dim=41, activation='relu'))  # discriminator takes 41 values from our dataset
    model.add(Dense(30, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # outputs 0 to 1, 1 being read and 0 being fake

    # model.summary()

    attack = Input(shape=(41,))
    validity = model(attack)

    return Model(attack, validity)

def build_generator():
    model = Sequential()
    model.add(Dense(100, input_dim=41))  # arbitrarily selected 100 for our input noise vector?
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(75))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(45))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(41, activation='relu'))  # outputs a generated vector of the same size as our data (41)

    # model.summary()

    noise = Input(shape=(41,))
    attack = model(noise)
    return Model(noise, attack)



if __name__ == '__main__':

    f = open("GANresults.txt", "w")
    f.write("""Results:
    -------------------------------""")
    f.close()


    dataframe = pd.read_csv('kdd_neptune_only_5000.csv').sample(4096) # sample 100 data points randomly from the csv
    batch_size = 512
    epochs = 10000

    # apply "le.fit_transform" to every column (usually only works on 1 column)
    le = LabelEncoder()
    dataframe_encoded = dataframe.apply(le.fit_transform)
    dataset = dataframe_encoded.values
    
    print("Real neptune attacks:")
    print(dataset[:2])

    # Set X as our input data and Y as our label
    X_train = dataset[:, 0:41].astype(float)
    Y_train = dataset[:, 41]
    
    np.savetxt("NeptuneFeatures.txt", X_train, fmt="%.0f")

    # size of each batch for training and number of epochs
    
    
    # labels for data. 1 for valid attacks, 0 for fake (generated) attacks
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))


    optimizer = Adam(0.0002, 0.5)
    discriminator = build_discriminator();
    discriminator.compile(loss='binary_crossentropy',
                               optimizer=optimizer,
                               metrics=['accuracy'])
    generator = build_generator()

    z = Input(shape=(41,))
    attack = generator(z)

    # discriminator.trainable = False

    validity = discriminator(attack)

    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    #stop training if our G loss starts going up (is G loss really the loss of the entire model?)
    loss_increase_count = 0;
    prev_g_loss = 0;
    


    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # selecting batch_size random attacks from our training data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        attacks = X_train[idx]

        # TODO: if there is problems, we might need to normalize our data too
        # generating an array of normalized random noise vectors
        noise = noise = np.random.normal(0, 1, (batch_size, 41))

        # create an array of generatid attack by using the model to predict based on the noise vectors
        gen_attacks = generator.predict(noise)

        # don't really understand what these loss functions are
        d_loss_real = discriminator.train_on_batch(attacks, valid)
        d_loss_fake = discriminator.train_on_batch(gen_attacks, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, 41))

        # Train the generator (to have the discriminator label samples as valid)
        g_loss = combined.train_on_batch(noise, valid)
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [Loss difference: %.3f, iterations with loss: %.0f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss, g_loss - prev_g_loss, loss_increase_count))

        
            
        # this is to break us out and save good results if our model starts increasing loss
        
        # if our generator loss icreased this iteration, increment the counter by 1
        if (g_loss - prev_g_loss) > 0:
            loss_increase_count = loss_increase_count + 1
        else: 
            loss_increase_count = 0  # otherwise, reset it to 0, we are still training effectively
            
        # if we've gone 10 iterations of increasing loss, stop training. Results are only saved every 50 iterations
        if loss_increase_count > 5:
            print('Stoping on iteration: ', epoch)
            break
        
        # write generated attacks to file every 50 iterations
        if epoch % 50 == 0:
            f = open("GANresults.txt", "a")
            rounded_attacks = np.around(gen_attacks, decimals=3)
            # np.savetxt("GANresults.txt", rounded_attacks, fmt="[%1.3f],")
            np.savetxt("GANresults.txt", gen_attacks, fmt="%.0f")
            f.close()
        
        prev_g_loss = g_loss
        
    results = np.loadtxt("GANresults.txt")
    
    print("Generated Neptune attacks: ")
    print(results[:2])

Using TensorFlow backend.


Real neptune attacks:
[[  0   0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0 225  16   1   2   0   0   6   2   0  18  16   7   3   0
    0   0   0   0   0   0]
 [  0   0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  36   6   1   2   0   0   7   4   0  18   6   3   1   0
    0   0   0   0   0   0]]
0 [D loss: 2.368061, acc.: 51.95%] [G loss: 0.751996] [Loss difference: 0.752, iterations with loss: 0]
1 [D loss: 1.741958, acc.: 41.70%] [G loss: 0.731671] [Loss difference: -0.020, iterations with loss: 1]
2 [D loss: 1.238004, acc.: 42.19%] [G loss: 0.726749] [Loss difference: -0.005, iterations with loss: 0]
3 [D loss: 0.790942, acc.: 54.10%] [G loss: 0.719540] [Loss difference: -0.007, iterations with loss: 0]
4 [D loss: 0.555188, acc.: 67.09%] [G loss: 0.710767] [Loss difference: -0.009, iterations with loss: 0]
5 [D loss: 0.471391, acc.: 70.12%] [G loss: 0.698561] [Loss difference: -0.012, iterations with loss: 0

75 [D loss: 0.973166, acc.: 50.10%] [G loss: 0.220845] [Loss difference: 0.003, iterations with loss: 0]
76 [D loss: 0.991612, acc.: 50.29%] [G loss: 0.224717] [Loss difference: 0.004, iterations with loss: 1]
77 [D loss: 0.959754, acc.: 50.10%] [G loss: 0.215526] [Loss difference: -0.009, iterations with loss: 2]
78 [D loss: 0.963391, acc.: 50.29%] [G loss: 0.222114] [Loss difference: 0.007, iterations with loss: 0]
79 [D loss: 0.977523, acc.: 50.00%] [G loss: 0.219163] [Loss difference: -0.003, iterations with loss: 1]
80 [D loss: 0.989238, acc.: 50.49%] [G loss: 0.218597] [Loss difference: -0.001, iterations with loss: 0]
81 [D loss: 0.980182, acc.: 50.10%] [G loss: 0.212645] [Loss difference: -0.006, iterations with loss: 0]
82 [D loss: 0.994857, acc.: 50.00%] [G loss: 0.207585] [Loss difference: -0.005, iterations with loss: 0]
83 [D loss: 1.013055, acc.: 50.10%] [G loss: 0.210751] [Loss difference: 0.003, iterations with loss: 0]
84 [D loss: 1.029735, acc.: 50.10%] [G loss: 0.208

153 [D loss: 1.206066, acc.: 50.00%] [G loss: 0.139695] [Loss difference: -0.004, iterations with loss: 0]
154 [D loss: 1.244004, acc.: 50.10%] [G loss: 0.138278] [Loss difference: -0.001, iterations with loss: 0]
155 [D loss: 1.219337, acc.: 50.10%] [G loss: 0.138162] [Loss difference: -0.000, iterations with loss: 0]
156 [D loss: 1.194682, acc.: 50.00%] [G loss: 0.147148] [Loss difference: 0.009, iterations with loss: 0]
157 [D loss: 1.252343, acc.: 50.10%] [G loss: 0.143584] [Loss difference: -0.004, iterations with loss: 1]
158 [D loss: 1.234421, acc.: 50.10%] [G loss: 0.141182] [Loss difference: -0.002, iterations with loss: 0]
159 [D loss: 1.170983, acc.: 50.00%] [G loss: 0.140741] [Loss difference: -0.000, iterations with loss: 0]
160 [D loss: 1.203303, acc.: 50.00%] [G loss: 0.142179] [Loss difference: 0.001, iterations with loss: 0]
161 [D loss: 1.176240, acc.: 50.10%] [G loss: 0.137990] [Loss difference: -0.004, iterations with loss: 1]
162 [D loss: 1.188465, acc.: 50.00%] [G

248 [D loss: 1.257536, acc.: 50.10%] [G loss: 0.125252] [Loss difference: 0.003, iterations with loss: 1]
249 [D loss: 1.310972, acc.: 50.00%] [G loss: 0.126777] [Loss difference: 0.002, iterations with loss: 2]
250 [D loss: 1.294180, acc.: 50.00%] [G loss: 0.122748] [Loss difference: -0.004, iterations with loss: 3]
251 [D loss: 1.320733, acc.: 50.00%] [G loss: 0.117281] [Loss difference: -0.005, iterations with loss: 0]
252 [D loss: 1.282281, acc.: 50.00%] [G loss: 0.119285] [Loss difference: 0.002, iterations with loss: 0]
253 [D loss: 1.334112, acc.: 50.00%] [G loss: 0.118269] [Loss difference: -0.001, iterations with loss: 1]
254 [D loss: 1.308635, acc.: 50.00%] [G loss: 0.116429] [Loss difference: -0.002, iterations with loss: 0]
255 [D loss: 1.301736, acc.: 50.00%] [G loss: 0.120404] [Loss difference: 0.004, iterations with loss: 0]
256 [D loss: 1.275321, acc.: 50.00%] [G loss: 0.117773] [Loss difference: -0.003, iterations with loss: 1]
257 [D loss: 1.299410, acc.: 50.00%] [G l

...

4343 [D loss: 0.743337, acc.: 50.00%] [G loss: 0.407883] [Loss difference: 0.007, iterations with loss: 0]
4344 [D loss: 0.747251, acc.: 49.90%] [G loss: 0.410177] [Loss difference: 0.002, iterations with loss: 1]
4345 [D loss: 0.749603, acc.: 50.49%] [G loss: 0.411241] [Loss difference: 0.001, iterations with loss: 2]
4346 [D loss: 0.734266, acc.: 50.68%] [G loss: 0.415315] [Loss difference: 0.004, iterations with loss: 3]
4347 [D loss: 0.741894, acc.: 50.88%] [G loss: 0.423766] [Loss difference: 0.008, iterations with loss: 4]
4348 [D loss: 0.738191, acc.: 50.78%] [G loss: 0.409152] [Loss difference: -0.015, iterations with loss: 5]
4349 [D loss: 0.744514, acc.: 50.78%] [G loss: 0.415801] [Loss difference: 0.007, iterations with loss: 0]
4350 [D loss: 0.755576, acc.: 49.80%] [G loss: 0.414736] [Loss difference: -0.001, iterations with loss: 1]
4351 [D loss: 0.734337, acc.: 50.39%] [G loss: 0.411794] [Loss difference: -0.003, iterations with loss: 0]
4352 [D loss: 0.750390, acc.: 50.6

4421 [D loss: 0.728691, acc.: 49.90%] [G loss: 0.427699] [Loss difference: 0.002, iterations with loss: 0]
4422 [D loss: 0.742820, acc.: 50.20%] [G loss: 0.427788] [Loss difference: 0.000, iterations with loss: 1]
4423 [D loss: 0.729387, acc.: 50.68%] [G loss: 0.426972] [Loss difference: -0.001, iterations with loss: 2]
4424 [D loss: 0.717471, acc.: 50.39%] [G loss: 0.427273] [Loss difference: 0.000, iterations with loss: 0]
4425 [D loss: 0.738473, acc.: 50.20%] [G loss: 0.423967] [Loss difference: -0.003, iterations with loss: 1]
4426 [D loss: 0.743257, acc.: 50.20%] [G loss: 0.437813] [Loss difference: 0.014, iterations with loss: 0]
4427 [D loss: 0.728800, acc.: 50.10%] [G loss: 0.428211] [Loss difference: -0.010, iterations with loss: 1]
4428 [D loss: 0.720931, acc.: 50.20%] [G loss: 0.433772] [Loss difference: 0.006, iterations with loss: 0]
4429 [D loss: 0.725268, acc.: 50.39%] [G loss: 0.432286] [Loss difference: -0.001, iterations with loss: 1]
4430 [D loss: 0.713550, acc.: 50.

4498 [D loss: 0.728607, acc.: 50.39%] [G loss: 0.443277] [Loss difference: 0.000, iterations with loss: 1]
4499 [D loss: 0.714404, acc.: 50.88%] [G loss: 0.443272] [Loss difference: -0.000, iterations with loss: 2]
4500 [D loss: 0.718959, acc.: 50.49%] [G loss: 0.444679] [Loss difference: 0.001, iterations with loss: 0]
4501 [D loss: 0.715781, acc.: 50.49%] [G loss: 0.437391] [Loss difference: -0.007, iterations with loss: 1]
4502 [D loss: 0.731092, acc.: 50.10%] [G loss: 0.438086] [Loss difference: 0.001, iterations with loss: 0]
4503 [D loss: 0.721137, acc.: 50.29%] [G loss: 0.438751] [Loss difference: 0.001, iterations with loss: 1]
4504 [D loss: 0.719325, acc.: 50.39%] [G loss: 0.444650] [Loss difference: 0.006, iterations with loss: 2]
4505 [D loss: 0.718845, acc.: 50.20%] [G loss: 0.451948] [Loss difference: 0.007, iterations with loss: 3]
4506 [D loss: 0.713138, acc.: 50.29%] [G loss: 0.433131] [Loss difference: -0.019, iterations with loss: 4]
4507 [D loss: 0.721843, acc.: 50.6

4576 [D loss: 0.744652, acc.: 51.07%] [G loss: 0.461091] [Loss difference: 0.010, iterations with loss: 0]
4577 [D loss: 0.693401, acc.: 50.98%] [G loss: 0.474095] [Loss difference: 0.013, iterations with loss: 1]
4578 [D loss: 0.701130, acc.: 50.49%] [G loss: 0.460004] [Loss difference: -0.014, iterations with loss: 2]
4579 [D loss: 0.706568, acc.: 50.98%] [G loss: 0.465252] [Loss difference: 0.005, iterations with loss: 0]
4580 [D loss: 0.711010, acc.: 50.78%] [G loss: 0.450913] [Loss difference: -0.014, iterations with loss: 1]
4581 [D loss: 0.697821, acc.: 50.78%] [G loss: 0.447889] [Loss difference: -0.003, iterations with loss: 0]
4582 [D loss: 0.707378, acc.: 50.78%] [G loss: 0.463195] [Loss difference: 0.015, iterations with loss: 0]
4583 [D loss: 0.701461, acc.: 50.98%] [G loss: 0.471028] [Loss difference: 0.008, iterations with loss: 1]
4584 [D loss: 0.703697, acc.: 50.78%] [G loss: 0.466959] [Loss difference: -0.004, iterations with loss: 2]
4585 [D loss: 0.695931, acc.: 50.

4653 [D loss: 0.829034, acc.: 50.88%] [G loss: 0.497449] [Loss difference: 0.029, iterations with loss: 0]
4654 [D loss: 0.692136, acc.: 50.49%] [G loss: 0.486896] [Loss difference: -0.011, iterations with loss: 1]
4655 [D loss: 0.707546, acc.: 50.49%] [G loss: 0.472732] [Loss difference: -0.014, iterations with loss: 0]
4656 [D loss: 0.768073, acc.: 50.10%] [G loss: 0.474328] [Loss difference: 0.002, iterations with loss: 0]
4657 [D loss: 0.702479, acc.: 50.88%] [G loss: 0.477513] [Loss difference: 0.003, iterations with loss: 1]
4658 [D loss: 0.749571, acc.: 50.10%] [G loss: 0.482983] [Loss difference: 0.005, iterations with loss: 2]
4659 [D loss: 0.700136, acc.: 50.88%] [G loss: 0.495197] [Loss difference: 0.012, iterations with loss: 3]
4660 [D loss: 0.689865, acc.: 51.07%] [G loss: 0.470371] [Loss difference: -0.025, iterations with loss: 4]
4661 [D loss: 0.766306, acc.: 50.59%] [G loss: 0.477344] [Loss difference: 0.007, iterations with loss: 0]
4662 [D loss: 0.697981, acc.: 50.7